In [149]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from datetime import timedelta
import matplotlib.dates as mdates
import sumolib
import osmnx as ox

import sys
import os
import math
import numpy as np

from fastsim import simdrive, vehicle, cycle
from fastsim import parameters as params

from mappymatch.constructs.geofence import Geofence
from mappymatch.utils.plot import plot_geofence
from mappymatch.maps.nx.nx_map import NxMap, NetworkType

import traci
import time
import csv

import xml.etree.ElementTree as ET
import xml.dom.minidom

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [6]:
print(os.environ['PATH'])

/common/software/install/migrated/anaconda/miniconda3_4.8.3-jupyter/bin:/sbin:/bin:/usr/sbin:/usr/bin:/opt/puppetlabs/bin:/opt/msi/sbin:/opt/msi/bin:/usr/local/bin:/usr/local/sbin:/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo/bin


In [7]:
os.environ['PATH'] += ":/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo/bin"

In [8]:
# Set SUMO_HOME
os.environ['SUMO_HOME'] = '/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo'

In [9]:
data_folder = "data/synthetic/Murphy"

In [10]:
csv_name = "TL5-218_2020W33_processed.csv"
file_name = os.path.join(data_folder, csv_name)

In [11]:
def read_data(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Convert string representations of lists or tuples back to lists or tuples
    # Use ast.literal_eval safely evaluate a string containing a Python literal or container display
    df['altitude_profile'] = df['altitude_profile'].apply(ast.literal_eval)
    df['velocity_profile'] = df['velocity_profile'].apply(ast.literal_eval)
    df['weight'] = df['weight'].apply(ast.literal_eval)
    df['total_fuel'] = df['total_fuel'].apply(ast.literal_eval)
    df['trajectory'] = df['trajectory'].apply(ast.literal_eval)
    df['matched_path'] = df['matched_path'].apply(ast.literal_eval)
    df['coordinate_id'] = df['coordinate_id'].apply(ast.literal_eval)
    df['road_id'] = df['road_id'].apply(ast.literal_eval)
    
    # Remove fractional seconds from 'time'
    df['trip_start_time'] = df['trip_start_time'].str.split('.').str[0]
    # Convert 'time' to datetime while ignoring fractional seconds
    df['trip_start_time'] = pd.to_datetime(df['trip_start_time'], format='%Y-%m-%d %H:%M:%S')

    df['trip_end_time'] = df['trip_end_time'].str.split('.').str[0]
    df['trip_end_time'] = pd.to_datetime(df['trip_end_time'], format='%Y-%m-%d %H:%M:%S')
    
    df['fastsim_velocity'] = df['fastsim_velocity'].apply(lambda x: [float(i) for i in x.split()])
    df['fastsim_power'] = df['fastsim_power'].apply(lambda x: [float(i) for i in x.split()])
    df['sumo_velocity'] = df['sumo_velocity'].apply(lambda x: [float(i) for i in x.split()])
    df['sumo_path'] = df['sumo_path'].str.split()
    return df

In [12]:
processed_df = read_data(file_name)

In [13]:
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity
0,2020-08-10 12:28:47,2020-08-10 13:13:17,2670.0,"[255.7, 255.2, 254.7, 254.2, 253.8, 252.9, 252...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[4.15, 4.15, 4.05, 4.1, 3.75, 3.8, 5.3, 6.25, ...",15.188,"[(44.788085, -93.462496), (44.78809, -93.46249...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1..."


In [14]:
net_file = 'data/Minneapolis.net.xml'
sumo_net = sumolib.net.readNet(net_file)

In [80]:
# Download the openstreet graph within the following bounding box (the same as map-matching):
# # bounding_box = {
# #         'min_latitude': 44.403672,
# #         'max_latitude': 45.450154,
# #         'min_longitude': -94.073366,
# #         'max_longitude': -92.696696
# #     }
# geojson_file = 'results/bounding_box_mn.geojson'
# geofence = Geofence.from_geojson(geojson_file)
# osmnx_net = ox.graph_from_polygon(
#         geofence.geometry, network_type='drive'
#     )

In [85]:
# save openstreet map:
# osm_file_path = "data/maps/minneapolis.graphml"
# ox.io.save_graphml(osmnx_net, osm_file_path)

In [15]:
# load openstreet map:
osm_file_path = "data/maps/minneapolis.graphml"
osmnx_net = ox.io.load_graphml(osm_file_path)

In [16]:
node_gdf, edge_gdf = ox.utils_graph.graph_to_gdfs(osmnx_net)

In [17]:
edge_gdf.loc[(34559009,33323569,0)]

osmid                                               202308024
name                                  Northeast Hoover Street
highway                                          unclassified
oneway                                                  False
reversed                                                 True
length                                                226.354
geometry    LINESTRING (-93.2185317 45.0007914, -93.218381...
ref                                                       NaN
lanes                                                       2
access                                                    NaN
maxspeed                                               20 mph
bridge                                                    NaN
junction                                                  NaN
tunnel                                                    NaN
area                                                      NaN
width                                                     NaN
Name: (3

In [18]:
node_gdf.loc[33323569]

y                                   44.998846
x                                  -93.217768
street_count                                4
highway                                  stop
ref                                       NaN
geometry        POINT (-93.2177684 44.998846)
Name: 33323569, dtype: object

In [19]:
node_gdf.loc[34559009]

y                                    45.000791
x                                   -93.218532
street_count                                 1
highway                                    NaN
ref                                        NaN
geometry        POINT (-93.2185317 45.0007914)
Name: 34559009, dtype: object

In [20]:
34559009 in node_gdf.index

True

In [87]:
# convert the sumo_path from the edge_id representation to (origin_node_id, dest_node_id) pair

# Function to convert edge list to node list
def edge_list_to_node_list(edges, sumo_net):
    # Split the edge list string into individual edges
    node_list = []
    for edge in edges:
        if edge.startswith(':'):
            node_list.append(node_list[-1])
        else:
            edge_obj = sumo_net.getEdge(edge)  # Get the edge object
            from_node_id = edge_obj.getFromNode().getID()  # Get the ID of the origin node
            to_node_id = edge_obj.getToNode().getID()  # Get the ID of the destination node
            node_tuple = (from_node_id, to_node_id, 0)  # Create a tuple 
            node_list.append(node_tuple)
    return node_list

processed_df['sumo_node_path'] = processed_df['sumo_path'].apply(lambda x: edge_list_to_node_list(x, sumo_net))
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity,sumo_node_path
0,2020-08-10 12:32:09,2020-08-10 13:13:17,2670.0,"[238.5, 238.2, 238.0, 237.9, 237.9, 237.7, 237...","[28.691, 28.73, 28.715, 28.34, 28.141, 27.859,...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[9.0, 8.3, 7.8, 6.15, 6.4, 6.3, 2.85, 0.55, 0....",15.188,"[(44.786858, -93.465484), (44.786858, -93.4655...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2...","[(8146751356, 188063558, 0), (8146751356, 1880..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1...","[(34559009, 33323569, 0), (34559009, 33323569,..."


In [22]:
# extract the valid road_id, total_fuel, velocity_profile, altitude_profile using the coordinate_id
# adjust the trip_start_time accordingly

def extracted_matched_edges(row):
    starting_index = row['coordinate_id'][0]
    row["trip_start_time"] = pd.to_datetime(row['trip_start_time']) + pd.to_timedelta(starting_index, unit='s')
    row['altitude_profile'] = row['altitude_profile'][starting_index:]
    row['velocity_profile'] = row['velocity_profile'][starting_index:]
    row['weight'] = [row['weight'][0]] * len(row['weight'][starting_index:])
    row['total_fuel'] = row['total_fuel'][starting_index:]
    row['trajectory'] = row['trajectory'][starting_index:]
    row['road_id'] = row['road_id'][starting_index:]
    return row
    
processed_df = processed_df.apply(extracted_matched_edges, axis = 1)
processed_df.head(2)

,trip_start_time,trip_end_time,travel_time,altitude_profile,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,fastsim_velocity,fastsim_power,sumo_path,sumo_velocity,sumo_node_path
0,2020-08-10 12:32:09,2020-08-10 13:13:17,2670.0,"[238.5, 238.2, 238.0, 237.9, 237.9, 237.7, 237...","[28.691, 28.73, 28.715, 28.34, 28.141, 27.859,...","[30000.0, 30000.0, 30000.0, 30000.0, 30000.0, ...","[9.0, 8.3, 7.8, 6.15, 6.4, 6.3, 2.85, 0.55, 0....",15.188,"[(44.786858, -93.465484), (44.786858, -93.4655...","[(8146751356, 188063558, 0), (188063558, 18805...","[202, 203, 204, 205, 206, 207, 208, 209, 210, ...","[(188055018, 9421214164, 0), (188055018, 94212...","[0.0, 5.686715111455639, 11.393352186998952, 1...","[0.0, 95.82343083537464, 229.88428843956595, 3...","[18237140#25, 18237140#25, 18237140#25, 182371...","[0.0, 5.686807374218478, 14.242079547476024, 2...","[(8146751356, 188063558, 0), (8146751356, 1880..."
1,2020-08-10 13:40:07,2020-08-10 13:57:35,1048.0,"[271.3, 272.8, 274.6, 276.8, 279.6, 281.9, 284...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8000.0, 8000.0, 8000.0, 8000.0, 8000.0, 8000....","[4.1, 4.05, 4.15, 4.1, 4.1, 4.25, 4.1, 4.1, 4....",16.875,"[(45.000419, -93.217756), (45.000417, -93.2177...","[(34559009, 33323569, 0), (33323569, 695998770...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(34559009, 33323569, 0), (34559009, 33323569,...","[0.0, 7.133007841291272, 12.904106082362334, 1...","[0.0, 130.41893294003683, 270.15292723619507, ...","[-202308024#14, -202308024#14, -202308024#14, ...","[0.0, 7.133123569088057, 15.077786642918364, 1...","[(34559009, 33323569, 0), (34559009, 33323569,..."


In [124]:
def index_range_in_sumo(edge_id, row, sumo_start_index):
    new_sumo_start_index = sumo_start_index - 1
    new_sumo_end_index = sumo_start_index - 1
    if sumo_start_index < len(row["sumo_node_path"]):
        for i in range(sumo_start_index, len(row["sumo_node_path"])):
            if str(edge_id[0]) in row["sumo_node_path"][i][0]:
                new_sumo_start_index = i
                break
        if new_sumo_start_index < sumo_start_index:
            return sumo_start_index, sumo_start_index + 1, False
        for i in range(new_sumo_start_index, len(row["sumo_node_path"])):
            if str(edge_id[1]) in row["sumo_node_path"][i][1] and (i >= len(row["sumo_node_path"]) - 1 or row["sumo_node_path"][i][1] != row["sumo_node_path"][i+1][1]):
                new_sumo_end_index = i
                return new_sumo_start_index, new_sumo_end_index + 1, True
    return sumo_start_index, sumo_start_index + 1, False

In [132]:
def initialize_edge_df():

    # Define the column names based on the keys from both dictionaries
    columns = [
        'trip_id', 'position', 'mass', 'elevation_change', 'energy_consumption_total_kwh',
        'simulated_energy_consumption_kwh', 'time', 'sumo_time', 'speed', 'sumo_speed', 'fastsim_speed',
        'time_acc', 'time_stage', 'week_day', 'tags', 'osmid', 'road_type', 'speed_limit',
        'length', 'lanes', 'bridge', 'endpoint_u', 'endpoint_v', 'direction_angle',
        'previous_orientation'
    ]

    # Initialize the DataFrame with these columns
    df_edges = pd.DataFrame(columns=columns)

    
    return df_edges

In [71]:
def cal_lanes(lanes_value):
    # Handle lists: assume you want to extract the first valid value
    if isinstance(lanes_value, list):
        # Filter out valid, numeric lane values, then take the first one
        valid_lanes = [lv for lv in lanes_value if pd.notna(lv) and str(lv).isdigit()]
        lanes_value = valid_lanes[0] if valid_lanes else 0

    # Handle floats and strings as before
    if pd.isna(lanes_value):
        return 0
    
    lanes_value_str = str(lanes_value)
    
    if lanes_value_str.isalpha():
        return 0
    
    if lanes_value_str.isalnum():
        # Extract digits and convert to integer, ensure non-negative
        numeric_part = ''.join(filter(str.isdigit, lanes_value_str))
        return int(numeric_part) if numeric_part and int(numeric_part) > 0 else 0
    else:
        # For non-alphanumeric strings, extract the first decimal value
        for i in lanes_value_str:
            if i.isdecimal():
                return int(i)
        return 0  # Default to 0 if no suitable value is found



def cal_bridge(bridge_value):
    # Handle lists: check each element
    if isinstance(bridge_value, list):
        # Initialize as not a bridge
        bridge_score = 0
        for item in bridge_value:
            if 'viaduct' in str(item).lower():
                bridge_score = max(bridge_score, 2)
            elif 'yes' in str(item).lower():
                bridge_score = max(bridge_score, 1)
        return bridge_score
    
    # Handle NaN values explicitly
    if pd.isna(bridge_value):
        return 0

    # Handle strings
    s = str(bridge_value).lower()  # Convert to string and lowercase to ensure consistency
    if 'viaduct' in s:
        return 2
    if 'yes' in s:
        return 1

    return 0  # Default return value if none of the conditions are met

    

In [74]:
def preprocess_edge(edge_gdf, node_gdf):
    # lanes
    edge_gdf['lanes_normed'] = edge_gdf['lanes'].apply(cal_lanes)
    edge_gdf['lanes_normed'] = edge_gdf['lanes_normed'].apply(lambda x: x if x <= 8 else 8)
    
    # bridges
    edge_gdf['bridge_normed'] = edge_gdf['bridge'].apply(lambda x: cal_bridge(x))
    
    # endpoints features
    edge_gdf['u'] = edge_gdf.index.get_level_values(0)
    edge_gdf['v'] = edge_gdf.index.get_level_values(1)
    edge_gdf['key'] = edge_gdf.index.get_level_values(2)
    edge_gdf['signal_u'] = edge_gdf.u.apply(lambda x: node_gdf.loc[x, 'highway']).fillna("None")
    edge_gdf['signal_v'] = edge_gdf.v.apply(lambda x: node_gdf.loc[x, 'highway']).fillna("None")
    list_uv = list(edge_gdf.signal_u.unique())+list(edge_gdf.signal_v.unique())
    endpoints_dictionary = dict()
    endpoints_dictionary['None'] = 0
    cnt_endpoint = 1
    for i in list_uv:
        if i not in endpoints_dictionary:
            endpoints_dictionary[i] = cnt_endpoint
            cnt_endpoint += 1
    np.save('results/endpoints_dictionary.npy', endpoints_dictionary)
    edge_gdf['signal_u_value'] = edge_gdf.signal_u.apply(lambda x: endpoints_dictionary[x])
    edge_gdf['signal_v_value'] = edge_gdf.signal_v.apply(lambda x: endpoints_dictionary[x])
    return edge_gdf

In [27]:
def highway_cal(network_seg):
    '''Calcuate the road type of an edge ('unclassified' in default)

    Args:
        Attributes of an edge
    
    Returns:
        Road type of the edge 
    '''
    if 'highway' in network_seg and network_seg['highway']:
        if isinstance(network_seg['highway'], str):
            return network_seg['highway']
        elif isinstance(network_seg['highway'], list):
            return network_seg['highway'][0]
    else:
        return 'unclassified'

In [105]:
def length_cal(network_seg):
    '''Calcuate the length of an edge (-1 in default)

    Args:
        Attributes of an edge
    
    Returns:
        length (m) of the edge 
    '''
    if 'length' not in network_seg:
        return -1
    
    if pd.isna(network_seg['length']):
        return -1
    else:
        return network_seg['length']

In [126]:
def speedlimit_cal(network_seg):
    '''Calculate the speed limit of an edge (default to 30*1.60934 km/h if unspecified)

    Args:
        network_seg: Attributes of an edge
    
    Returns:
        Speed limit (km/h) of the edge 
    '''
    # Check if 'maxspeed' attribute exists and has a non-empty value
    if 'maxspeed' in network_seg and network_seg['maxspeed']:
        maxspeed = network_seg['maxspeed']
        # Handle different types of 'maxspeed' values
        if isinstance(maxspeed, list):
            # Assume the first element of the list is the relevant speed limit
            maxspeed = maxspeed[0]
        
        if isinstance(maxspeed, str):
            # Extract numeric part of the string
            numeric_speed = ''.join(filter(str.isdigit, maxspeed))
            if numeric_speed:
                # Convert to int and adjust to km/h
                return int(numeric_speed) * 1.60934
        
        elif isinstance(maxspeed, (int, float)):
            if not pd.isna(maxspeed):
                # Directly use the numeric value, assuming it's in mph and convert to km/h
                return maxspeed * 1.60934
    
    # Fallbacks based on highway type if 'maxspeed' is not specified
    elif highway_cal(network_seg) == "motorway":
        return 55 * 1.60934
    elif highway_cal(network_seg) == "motorway_link":
        return 50 * 1.60934
    
    # Default speed limit if none of the above conditions are met
    return 30 * 1.60934


In [30]:
def compute_direction_angle(network_seg):
    xs, ys = network_seg['geometry'].xy
    latitude_o,longitude_o = xs[0],ys[0]
    latitude_d,longitude_d  = xs[-1],ys[-1]
    direction = [latitude_d-latitude_o,longitude_d-longitude_o]
    direction_array = np.array(direction)
    cosangle = direction_array.dot(np.array([1,0]))/(np.linalg.norm(direction_array)) 
    if np.cross(direction_array,np.array([1,0])) < 0:
        direction_angle = math.acos(cosangle)*180/np.pi
    else :
        direction_angle = -math.acos(cosangle)*180/np.pi
    return direction_angle

In [31]:
def ori_cal(coor_a,coor_b,coor_c):
    '''Calcuate the orientation change from vector ab to bc (0 in default, right turn > 0, left turn < 0)

    10 degree is the threshold of a turn.

    Args:
        coor_a: coordinate of point a
        coor_b: coordinate of point b
        coor_c: coordinate of point c
    
    Returns:
        's': straight
        'l': left-hand turn
        'r': right-hand turn
    '''
    a = np.array(coor_a)
    b = np.array(coor_b)
    c = np.array(coor_c)
    v_ab = b-a
    v_bc = c-b
    cosangle = v_ab.dot(v_bc)/(np.linalg.norm(v_bc) * np.linalg.norm(v_ab)+1e-16) 
    return math.acos(cosangle)*180/np.pi if np.cross(v_ab,v_bc) < 0 else -math.acos(cosangle)*180/np.pi

def previous_orientation_cal(network_seg, edge_gdf, prev_edge_id):
    if prev_edge_id == -1 :
        orientation = 0
    elif 'geometry' in network_seg and network_seg['geometry'] and edge_gdf.loc[prev_edge_id,'geometry']:
        xs, ys = network_seg['geometry'].xy
        #print(id_before)
        xl, yl = edge_gdf.loc[prev_edge_id,'geometry'].xy
        coor_b = [xs[0],ys[0]]
        coor_c = [xs[1],ys[1]]
        coor_a = [xl[-2],yl[-2]]
        #print(beg,coor_a,coor_b,coor_c)
        orientation = ori_cal(coor_a,coor_b,coor_c)
    else:
        orientation = 0
    return orientation

In [106]:
def extract_osm_edge_feature(edge_gdf, edge_id, prev_edge_id):
    osm_edge_feature = edge_gdf.loc[edge_id]
    edge_features = {
        'tags': edge_id,
        'osmid': osm_edge_feature.osmid,
        'road_type': highway_cal(osm_edge_feature),
        'speed_limit': speedlimit_cal(osm_edge_feature),
        'length': length_cal(osm_edge_feature),
        'lanes': osm_edge_feature.lanes_normed,
        'bridge': osm_edge_feature.bridge_normed,
        'endpoint_u': osm_edge_feature.signal_u_value, 
        'endpoint_v': osm_edge_feature.signal_v_value,
        'direction_angle': compute_direction_angle(osm_edge_feature),
        'previous_orientation': previous_orientation_cal(osm_edge_feature, edge_gdf, prev_edge_id)
    }
    return edge_features

In [134]:
# for a node pair in road_id, if that both the origin node and the dest node exist in sumo_path, then it is a valid edge for training 
df_edges = initialize_edge_df()

for i in range(len(processed_df)):
    row = processed_df.iloc[i]
    # these indics are exclusive: [start_index, end_index]
    trip_start_index = 0
    trip_end_index = 0
    
    # if the vehicle is not moving at the start of the trip, skip these data loggings.
    if row['velocity_profile'][trip_start_index] <= 1e-10:
        trip_start_index += 1
        trip_end_index += 1
        
    sumo_start_index = 0
    sumo_end_index = 0
    
    position_of_edge = 0
    edge_gdf = preprocess_edge(edge_gdf, node_gdf)
    # output dataframe where each row represent an edge
    
    
    prev_edge_id = -1
    
    
    
    while trip_start_index < len(row["road_id"]):
              
        edge_id = row["road_id"][trip_start_index]
        # validate edge_id by checking if both the origin and the destinaiton occur in the sumo_node_path
        
        # exclusive index
        while trip_end_index < len(row["road_id"]) and row["road_id"][trip_end_index] == edge_id:
            trip_end_index += 1
            
        # exclusive index
        sumo_start_index, sumo_end_index, found_flag = index_range_in_sumo(edge_id, row, sumo_start_index)
                
        # if the edge_id is in the sumo simulation
        if found_flag:
            
            # extract simulated_features
            simulated_features = {
                'trip_id': (0, i),
                'position': position_of_edge,
                'mass': row['weight'][trip_start_index],
                'elevation_change': row['altitude_profile'][trip_end_index-1] - row['altitude_profile'][trip_start_index],
                'energy_consumption_total_kwh': sum(row['total_fuel'][trip_start_index:trip_end_index])*40.3/3600/3.7854,
                'simulated_energy_consumption_kwh': sum(row['fastsim_power'][sumo_start_index:sumo_end_index])/3600,
                'time': trip_end_index - trip_start_index,
                'sumo_time': sumo_end_index-sumo_start_index,
                'speed': row['velocity_profile'][trip_start_index:trip_end_index],
                'sumo_speed': row['sumo_velocity'][sumo_start_index:sumo_end_index],
                'fastsim_speed': row['fastsim_velocity'][sumo_start_index:sumo_end_index],
                'time_acc': row['trip_start_time'],
                'time_stage': row['trip_start_time'].hour//4 + 1,
                'week_day': row['trip_start_time'].weekday()+1,
                }
            edge_features = extract_osm_edge_feature(edge_gdf, edge_id, prev_edge_id)
            
            # Combine edge_features and additional_features
            edge_row = {**edge_features, **simulated_features}

            # Add the combined features as a new row to the DataFrame
            df_edges.loc[len(df_edges)] = edge_row
            
            # update the sumo_start_index search range
            sumo_start_index = sumo_end_index
            position_of_edge += 1
            
        prev_edge_id = edge_id
        
        # update the trip_start_index anyway
        trip_start_index = trip_end_index
        
 

In [152]:
mean_true = df_edges['energy_consumption_total_kwh'].mean()
mean_pred = df_edges['simulated_energy_consumption_kwh'].mean()

print(f'Mean (True): {mean_true}')
print(f'Mean (Predicted): {mean_pred}')


# Assuming df_edges is your pandas DataFrame
y_true = df_edges['energy_consumption_total_kwh']
y_pred = df_edges['simulated_energy_consumption_kwh']

# Compute MSE
mse = mean_squared_error(y_true, y_pred)

print(f'Mean Squared Error: {mse}')



mae = mean_absolute_error(df_edges['energy_consumption_total_kwh'], df_edges['simulated_energy_consumption_kwh'])

print(f'Mean Absolute Error (MAE): {mae}')



Mean (True): 1.774352855364359
Mean (Predicted): 2.121706154077607
Mean Squared Error: 5.431228247360808
Mean Absolute Error (MAE): 1.1593115340260198
